In [1]:
from enum import IntEnum

from pydantic import Field
import numpy as np

from spycekit import DesignBase, FeatureSpace, Population

In [2]:
import json

class Style(IntEnum):
    Cool = 0
    Fast = 1

class MyDesign(DesignBase):
    length: float = Field(..., ge=0.5, le=2.5)
    width: float = Field(..., ge=3.5, le=4.5)
    style: Style

d = MyDesign(length=0.5, width=4, style=Style.Cool)
sim = MyDesign.sample_once()
print(sim.to_series())

# Similar, but the latter can be used with datamodel-codegen
# Option 1: Convert the Design to a FeatureSpace, then dump the feature space
featurespace = MyDesign.to_featurespace()
serialized_featurespace = featurespace.model_dump_json(indent=4)
print(serialized_featurespace)

# Option 2:
# Dump the JSON Schema of the design
serialized_model_schema = json.dumps(MyDesign.model_json_schema(), indent=4)
print(serialized_model_schema)

# Deserialization
# Option 1
# Reconstruct a class which is identical to the design (up to validation) from a feature space representation!
# Assume you have already loaded the featurespace using standard pydantic deserializer
# NB: Option 2 would be using json schema and data model codegen
NewModel = featurespace.to_designmodel()
d = NewModel(length=1, width=4, style=Style.Cool)
d.style = Style.Fast
print(NewModel.sample_once())


id        8c0e472e-a459-4b7c-a70e-06faa19f3ad6
length                                1.096548
width                                  3.92093
style                               Style.Fast
dtype: object
{
    "name": "MyDesign",
    "id": "a9f5d1a4-71a5-437f-8a9b-afa49937126a",
    "features": {
        "length": {
            "fieldname": "length",
            "bounds": [
                0.5,
                2.5
            ],
            "mode": "Continuous",
            "min": 0.5,
            "max": 2.5
        },
        "width": {
            "fieldname": "width",
            "bounds": [
                3.5,
                4.5
            ],
            "mode": "Continuous",
            "min": 3.5,
            "max": 4.5
        },
        "style": {
            "fieldname": "style",
            "bounds": {
                "Cool": 0,
                "Fast": 1
            },
            "mode": "Categorical",
            "min": null,
            "max": null
        }
    }
}
{
   

In [24]:
pop = Population.from_feature_space(featurespace,n=10)

group,features
0,4.308816
1,4.368784
2,3.971923
3,3.798967
4,3.891401
5,4.287942
6,4.144749
7,3.880368
8,3.932661
9,4.494770


In [8]:
pop = Population.from_feature_space(featurespace,n=10,index_by="all", flatten=True)
pop.data["Result"] = np.random.rand(len(pop.data))

# setting by a vector
pop.data.loc[pop.data.iloc[0].name,"Result"] = 3
id = pop.data.index.to_frame(index=False).loc[0,"id"]

pop.data.xs(key=id,level="id",drop_level=False)
pop.data.xs(key=1,level="style",drop_level=False)

Result
space_id                                id                                      length   width    style          
225915896070189228449530361031937167978 202477620926079631231575735006206790515 1.260392 3.585269 1      0.771806
                                        154597833798282268221578268671599431281 0.687892 4.384440 1      0.049196
                                        296098407725868805030630036587595072867 1.981690 4.110407 1      0.570965

In [18]:
pop = Population.from_feature_space(featurespace,n=10,index_by="features", flatten=True)
pop.data["Result"] = np.random.rand(len(pop.data))
pop.data.loc[pop.data.iloc[0].name,"Result"] = 3
pop.data

C:\Users\szvsw\AppData\Local\Temp\ipykernel_40240\1035819227.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  pop.data.loc[pop.data.iloc[0].name,"Result"] = 3


,,field,space_id,id,Result
length,width,style,,,
2.102487,3.521930,0,225915896070189228449530361031937167978,147106845120632615570322959845117767909,3.000000
1.212761,3.712750,0,225915896070189228449530361031937167978,8821603650716012200842498111698990902,0.210913
1.284772,3.868100,1,225915896070189228449530361031937167978,57837951129698772973657901408207948707,0.836649
2.067309,3.709227,0,225915896070189228449530361031937167978,21279863025776447401115572519628133319,0.668853
1.264237,3.561250,1,225915896070189228449530361031937167978,224211056147351215818993058509594089389,0.546197
1.429411,3.989054,0,225915896070189228449530361031937167978,118573731346841273868121600467472636515,0.725233
0.873520,3.864592,1,225915896070189228449530361031937167978,41333908097194962507400800161979991216,0.294809
1.589031,3.958323,1,225915896070189228449530361031937167978,75081517650170848008755200379401547011,0.145420
2.136121,4.014234,0,225915896070189228449530361031937167978,316052337725197525918107693668005845808,0.855402


In [41]:
df = Population.from_feature_space(featurespace,n=0, index_by_metadata=True).data

group       features            
field         length width style
space_id id                     
al       b       0.0   1.0   2.0

In [14]:
import pandas as pd
pop = featurespace.make_population(5, index_by_metadata=False)

# df = pd.DataFrame(data={"a": range(5)},index=pd.MultiIndex.from_frame(pop.data) )
df = pd.DataFrame(index=pd.MultiIndex.from_frame(pop.data).names)

df, pop.data.columns.names, pop.data.columns.get_level_values(0), pop.data.columns.get_level_values(1)
# pop.data.columns = pop.data.columns.to_flat_index()

# Transform to a multiindexable series
df2 = pop.data.set_index(pop.data["metadata"]["id"])
df2.unstack()["features"]["length"]

# From  hierarchical to flat columns
pop.data.columns = pop.data.columns.to_flat_index()
pop.data

# from flat to heierarchical
pop.data.columns = pd.MultiIndex.from_tuples(pop.data.columns, names=["group","field"])


df = pop.data.set_index([("metadata","space_id"),("metadata","id")])


# print(pop.data.columns.to_flat_index())

# Metadata df = pd.DataFrame(index=pop.data.columns)

# import pandas as pd
# pd.DataFrame(data={"a": range(5)}, index=pd.MultiIndex.from_frame(pop.data))

# pop.data.reset_index(level="Label", names=("metadata","space_id"))

group                                                                            features  \
field                                                                              length   
(metadata, space_id)                    (metadata, id)                                      
290517938470675618075255113245515151027 182671076918680374034869685552947662223  1.042530   
                                        85356800373463795566448449910268079589   0.909280   
                                        212656097656733472273140491868840272154  0.847764   
                                        154990671846313946732284640716691465965  1.927589   
                                        69484431114681991924306424400287148706   0.891419   

group                                                                                      \
field                                                                               width   
(metadata, space_id)                    (metadata, id)                                      
290517938470675618075255113245515151027 182671076918680374034869685552947662223  3.590545   
                                        85356800373463795566448449910268079589   3.548912   
                                        212656097656733472273140491868840272154  4.357612   
                                        154990671846313946732284640716691465965  3.896172   
                                        69484431114681991924306424400287148706   4.309041   

group                                                                                  
field                                                                           style  
(metadata, space_id)                    (metadata, id)                                 
290517938470675618075255113245515151027 182671076918680374034869685552947662223     1  
                                        85356800373463795566448449910268079589      0  
                                        212656097656733472273140491868840272154     1  
                                        154990671846313946732284640716691465965     1  
                                        69484431114681991924306424400287148706      1

group                                                                            features
(metadata, space_id)                    (metadata, id)                                   
290517938470675618075255113245515151027 182671076918680374034869685552947662223  3.590545
                                        85356800373463795566448449910268079589   3.548912
                                        212656097656733472273140491868840272154  4.357612
                                        154990671846313946732284640716691465965  3.896172
                                        69484431114681991924306424400287148706   4.309041